### Basic Imports

In [29]:
import sensemakr as smkr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn import preprocessing
from datetime import datetime
from dateutil.parser import parse
from statsmodels.iolib.summary2 import summary_col

### File Read

In [10]:
df = pd.read_csv('BPIC17_O_Accepted.csv',sep=";")

### Data Preprocessing

In [23]:
def read_prep_data(file):
    df = pd.read_csv(r"%s" % file, sep=';')
    # missining selected
    df = df[df['Selected'] != 'missing']  # make sure that we have data for selected attribute

    # CreditScore handeling
    df.CreditScore.replace(0.0, np.nan, inplace=True)
    df['CreditScore'].fillna((df['CreditScore'].median()), inplace=True)

    # MonthlyCost handeling
    df.MonthlyCost.replace(0.0, np.nan, inplace=True)
    df['MonthlyCost'].fillna((df['MonthlyCost'].median()), inplace=True)

    # NumberOfTerms handeling
    df.NumberOfTerms.replace(0.0, np.nan, inplace=True)
    df['NumberOfTerms'].fillna((df['NumberOfTerms'].median()), inplace=True)

    # FirstWithdrawalAmount handeling
    df.FirstWithdrawalAmount.replace(0.0, np.nan, inplace=True)
    df['FirstWithdrawalAmount'].fillna((df['FirstWithdrawalAmount'].median()), inplace=True)

    # map Selected to 1(signed), and 0(not signed)
    df['Selected'] = df['Selected'].map({'True': 1, 'False': 0})

    # for FrequencyOfIncompleteness
    #     df2 = df[df['Activity']=='A_Incomplete']
    #     df_new = pd.DataFrame(df2.groupby(['Case ID'])['Activity'].count()).reset_index()
    #     df_new.columns = ['Case ID', 'FrequencyOfIncompleteness']
    #     df_new = pd.DataFrame(df_new.groupby('Case ID')['FrequencyOfIncompleteness'].sum()).reset_index()
    #     df = pd.merge(df_new, df, on='Case ID')

    # For NumberOfOffers
    df2 = df[df['Activity'] == "O_Created"]  # to count offers
    df_new = pd.DataFrame(df2.groupby(['Case ID'])['Activity'].count()).reset_index()
    df_new.columns = ['Case ID', 'NumberOfOffers']
    df = pd.merge(df_new, df, on='Case ID')

    # For matchRequested
    df['MatchedRequest'] = np.where((df.RequestedAmount <= df.OfferedAmount), 'True', 'False')

    df = df.groupby('Case ID').apply(get_duration)
    df = df.reset_index(drop=True)

    # O_Sent (mail and online)
    #     df2 = df[df['Activity'] == 'O_Sent (mail and online)'] # to count offers
    #     df_new = pd.DataFrame(df2.groupby(['Case ID'])['Activity'].count()).reset_index()
    #     df_new.columns = ['Case ID', 'O_sent_mail_online_frequency']
    #     df = pd.merge(df_new, df, on='Case ID')

    # O_Sent (online only)
    #     df2 = df[df['Activity'] == 'O_Sent (online only)'] # to count offers
    #     df_new = pd.DataFrame(df2.groupby(['Case ID'])['Activity'].count()).reset_index()
    #     df_new.columns = ['Case ID', 'O_sent_online_only_frequency']
    #     df = pd.merge(df_new, df, on='Case ID')

    # binning columns
    #    df['new_duration'] = pd.cut(df['durationDays'], [0,8,15,30,31,168],
    #                                include_lowest=True, right=False, labels=['0-7','8-14','15-29','30','31+'])
    #    df['new_duration'] = df['new_duration'].astype(str)
    #    df['new_FrequencyOfIncompleteness'] = pd.cut(df['FrequencyOfIncompleteness'], 15)
    #    df['new_FrequencyOfIncompleteness'] = df['new_FrequencyOfIncompleteness'].astype(str)xs

    df['binned_RequestedAmount'] = pd.qcut(df['RequestedAmount'], 5, labels=['0-5000', '5001-10000',
                                                                             '10001-15000', '15001-25000', '25000+'])
    df['binned_RequestedAmount'] = df['binned_RequestedAmount'].astype(str)

    df['binned_duration'] = pd.qcut(df['durationDays'], 5, labels=['0-8', '9-13', '14-22', '23-30', '30+'])
    df['binned_duration'] = df['binned_duration'].astype(str)

    df['binned_NoOfTerms'] = pd.qcut(df['NumberOfTerms'], 5, labels=['6-48', '49-60', '61-96', '97-120', '120+'])
    df['binned_NoOfTerms'] = df['binned_NoOfTerms'].astype(str)

    df['binned_CreditScore'] = pd.qcut(df['CreditScore'], 2, labels=['low', 'high'])
    df['binned_CreditScore'] = df['binned_CreditScore'].astype(str)

    df['binned_MonthlyCost'] = pd.qcut(df['MonthlyCost'], 5, labels=['40-148', '149-200', '201-270',
                                                                     '271-388', '388+'])
    df['binned_MonthlyCost'] = df['binned_MonthlyCost'].astype(str)

    df['binned_FirstWithdrawalAmount'] = pd.qcut(df['FirstWithdrawalAmount'], 3,
                                                 labels=['0-7499', '7500-9895', '9896-75000'])
    df['binned_FirstWithdrawalAmount'] = df['binned_FirstWithdrawalAmount'].astype(str)

    df['binned_NumberOfOffers'] = pd.cut(df['NumberOfOffers'], [1, 2, 3, 11],
                                         include_lowest=True, right=False, labels=['1', '2', '3+'])
    df['binned_NumberOfOffers'] = df['binned_NumberOfOffers'].astype(str)

    df = df.groupby('Case ID').apply(keep_last)
    df = df.reset_index(drop=True)

    # lower case
    column = ['ApplicationType', 'LoanGoal', 'MatchedRequest']
    for col in column:
        df[col] = df[col].str.lower()

    return df

def keep_last(group):
    return group.tail(1)


def get_duration(gr):
    df = pd.DataFrame(gr)
    if len(df[(df["Activity"] == "A_Denied") | (df["Activity"] == "A_Cancelled") | (
            df["Activity"] == "A_Pending")]) > 0:
        df['new_date'] = [datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in df['time:timestamp']]

        first_dt = df[df['Activity'] == 'O_Create Offer']['new_date']
        last_dt = \
            df[(df["Activity"] == "A_Denied") | (df["Activity"] == "A_Cancelled") | (df["Activity"] == "A_Pending")][
                'new_date']

        first_dt = first_dt[first_dt.index.values[0]]
        # print(last_dt)
        last_dt = last_dt[last_dt.index.values[0]]

        d1 = parse(str(first_dt))
        d2 = parse(str(last_dt))

        delta_days = (d2 - d1).days
        # print(delta_days,'\n')
        df['durationDays'] = delta_days
        return df


In [24]:
data = 'BPIC17_O_Accepted.csv'
df = read_prep_data(data)

### Label Encoding few of the features as the regression model works with numerical data

In [25]:
le = preprocessing.LabelEncoder()
object_cols = ['ApplicationType','RequestedAmount','label','LoanGoal','Activity']
for col in object_cols:
      df[col] = le.fit_transform(df[col])

In [26]:
pd.set_option('display.max_columns', None)
df.head(10)

,Case ID,NumberOfOffers,ApplicationType,LoanGoal,RequestedAmount,label,Activity,org:resource,Action,EventOrigin,lifecycle:transition,Accepted,Selected,FirstWithdrawalAmount,MonthlyCost,NumberOfTerms,OfferedAmount,CreditScore,timesincelastevent,timesincecasestart,timesincemidnight,event_nr,month,weekday,hour,open_cases,time:timestamp,MatchedRequest,new_date,durationDays,binned_RequestedAmount,binned_duration,binned_NoOfTerms,binned_CreditScore,binned_MonthlyCost,binned_FirstWithdrawalAmount,binned_NumberOfOffers
0,Application_1000086665,1,1,8,7,1,3,User_1,Deleted,Workflow,ate_abort,True,0,5000.0,241.28,22.0,5000.0,899.0,0.001067,46923.253667,360,22,9,0,6,2271,2016-09-05 06:00:36.893,true,2016-09-05 06:00:36.893,30,0-5000,23-30,6-48,low,201-270,0-7499,1
1,Application_1000158214,1,1,4,147,0,8,User_90,Deleted,Workflow,ate_abort,True,1,8304.0,250.00,57.0,12500.0,929.0,0.000067,11567.573967,662,25,6,4,11,1659,2016-06-10 11:02:01.282,true,2016-06-10 11:02:01.282,4,10001-15000,0-8,49-60,high,201-270,7500-9895,1
2,Application_1000311556,1,1,0,575,1,3,User_1,Deleted,Workflow,ate_abort,True,0,8304.0,500.00,111.0,45000.0,899.0,0.000217,44044.188133,360,18,5,3,6,1533,2016-05-05 06:00:48.963,true,2016-05-05 06:00:48.963,30,25000+,23-30,97-120,low,388+,7500-9895,1
3,Application_1000334415,1,1,8,7,1,8,User_68,Deleted,Workflow,ate_abort,True,1,5000.0,152.82,36.0,5000.0,899.0,0.000150,19626.277183,465,40,9,3,7,2347,2016-09-29 07:45:34.389,true,2016-09-29 07:45:34.389,12,0-5000,9-13,6-48,low,149-200,0-7499,1
4,Application_1000339879,1,1,2,515,0,8,User_87,Deleted,Workflow,ate_abort,True,1,8304.0,375.00,126.0,37500.0,798.0,0.000067,18494.640683,551,51,3,2,9,1644,2016-03-30 09:11:48.600,true,2016-03-30 09:11:48.600,12,25000+,9-13,120+,low,271-388,7500-9895,1
5,Application_100034150,1,1,2,7,0,8,User_119,Deleted,Workflow,ate_abort,False,1,577.0,56.30,120.0,5000.0,824.0,0.000033,16038.006067,695,55,3,1,11,1668,2016-03-08 11:35:09.066,true,2016-03-08 11:35:09.066,10,0-5000,9-13,97-120,low,40-148,0-7499,1
6,Application_1000386745,1,1,0,7,0,4,User_123,Deleted,Workflow,ate_abort,True,1,5000.0,100.25,58.0,5000.0,1080.0,0.000067,15807.465350,838,46,12,1,13,1801,2016-12-06 13:58:37.773,true,2016-12-06 13:58:37.773,10,0-5000,9-13,49-60,high,40-148,0-7499,1
7,Application_1000474975,1,1,8,609,0,8,User_68,Deleted,Workflow,complete,True,1,50000.0,923.56,60.0,50000.0,893.0,0.000117,18449.561100,666,37,6,2,11,2016,2016-06-29 11:06:58.078,true,2016-06-29 11:06:58.078,12,25000+,9-13,49-60,low,388+,9896-75000,1
8,Application_1000557783,1,1,4,107,0,8,User_30,Deleted,Workflow,complete,True,1,10000.0,213.21,53.0,10000.0,805.0,0.000100,18361.051150,789,27,3,1,13,1649,2016-03-22 13:09:48.407,true,2016-03-22 13:09:48.407,12,5001-10000,9-13,49-60,low,201-270,7500-9895,1
9,Application_1000604502,1,1,8,107,1,3,User_1,Deleted,Workflow,withdraw,True,0,10000.0,230.66,48.0,10000.0,899.0,0.000117,48107.014983,420,23,1,0,7,982,2017-01-09 07:00:24.768,true,2017-01-09 07:00:24.768,30,5001-10000,23-30,6-48,low,201-270,7500-9895,1


### Creating a Regression model

We run the following linear regression model, where we regress Selected Loan applications on Credit score
further adjusting for Requested Amount, No of Terms etc. Here we run this regression using statsmodel.



In [28]:
reg_model2 = smf.ols(formula='Selected ~ ApplicationType + CreditScore + LoanGoal + Activity + RequestedAmount + NumberOfTerms +  FirstWithdrawalAmount + MonthlyCost',data=df)
                     
loan_model = reg_model2.fit()                                     

### Summary of regression results for No of terms (the coefficient of regression)

In [38]:
summary_col(loan_model, regressor_order=["NumberOfTerms"], drop_omitted=True)

,Selected
NumberOfTerms,0.0009
,(0.0001)
R-squared,0.2902
R-squared Adj.,0.2900


### Sense maker analyses for Confounding effect

The main arguments of the call are:

model: the OLS model with the outcome regression. In our case, loan_model.

treatment: the name of the treatment variable. In our case, “NumberofTerms”.

benchmark_covariates: the names of covariates that will be used to bound the plausible strength of the unobserved confounders. Here, we put “CreditScore”, which is arguably one of the main determinants of Loan Application being accepted or not

kd and ky: these arguments parameterize how many times stronger the confounder is related to the treatment (kd) and to the outcome (ky) in comparison to the observed benchmark covariates (in this case, Credit Score). In our example, setting kd = [1, 2, 3] and ky = [1,2,3] means we want to investigate the maximum strength of a confounder once, twice, or three times as strong as Credit Score (in explaining treatment and outcome variation). If only kd is given, ky will be set equal to kd.

q: fraction of the effect estimate that would have to be explained away to be problematic. Setting q = 1, means that a reduction of 100% of the current effect estimate, that is, a true effect of zero, would be deemed problematic. The default is 1.

alpha: significance level of interest for statistical inference. The default is 0.05.

reduce: should we consider confounders acting towards increasing or reducing the absolute value of the estimate? The default is reduce = True, which means we are considering confounders that pull the estimate towards (or through) zero.

In [39]:
loan_sense =  smkr.Sensemakr(model = loan_model,
                              treatment = "NumberOfTerms",
                              benchmark_covariates = ["CreditScore"],
                              kd = [1,2,3],
                              ky = [1,2,3],
                              q = 1.0,
                              alpha = 0.05,
                              reduce = True)

### Code Output for a latex or html table 


These three sensitivity statistics provide a minimal reporting for sensitivity analysis. More precisely:

The robustness value for bringing the point estimate of NumberOfTerms exactly to zero (RV{q=1}) is 6% . 

This means that unobserved confounders that explain 6% of the residual variance both of the treatment 

and of the outcome are sufficiently strong to explain away all the observed effect. 

On the other hand, unobserved confounders that do not explain at least 6% of the residual variance both of the treatment and of the outcome are not sufficiently strong to do so.


The robustness value for testing the null hypothesis that the coefficient of NumberOfTerms is zero (RV{q=1,alpha = 0.05}) falls to 4.9%. 

This means that unobserved confounders that explain 4.9% of the residual variance both of the treatment and of the outcome are sufficiently strong to bring the lower bound of the confidence interval to zero (at the chosen significance level of 5%). 

On the other hand, unobserved confounders that do not explain at least 4.9% of the residual variance both of the treatment and of the outcome are not sufficiently strong to do so.


Finally, the partial R^2 of NumberOfTerms with Selected means that, in an extreme scenario, in which we assume that unobserved confounders explain all of the left out variance of the outcome, these unobserved confounders would need to explain at least 0.4% of the residual variance of the treatment to fully explain away the observed effect.


The lower corner of the table, further provides bounds on the strength of an unobserved confounder as strong as the observed covariate CreditScore, resulting in R^2{Y->Z|X, D} = 0.1% and R^2{D->Z|X} = 0.1% . Since both of those are below the RV of 6.0%, we conclude confounders as strong as NumberOfTerms, in explaining treatment and outcome variations, are not sufficiently strong to explain away the observed estimate.

Moreover, the bound of R^2{D->Z|X} = 0.1\% is below the partial 𝑅^2 of the treatment with the outcome, R^2{Y~D given X} = 0.4\%, this means that even an extreme confounder explaining all residual variation of the outcome, and as strongly associated with the treatment as NumberOfTerms would not be able to overturn the research conclusions.

In [42]:
html_code = loan_sense.ovb_minimal_reporting(format = "html")


Outcome: Selected 
 
 
 Treatment 
 Est. 
 S.E. 
 t-value 
 R 2 Y~D|X 
 RV q = 1.0 
 RV q = 1.0, α = 0.05 
 
 
 
 
 NumberOfTerms 
 0.001 
 0.0 
 10.9 
 0.4% 
 6.0% 
 4.9% 
 
 
 
 Note: df = 31402; Bound ( 1x CreditScore ): R 2 Y~Z|X,D = 0.1%, R 2 D~Z|X =0.1%